In [1]:
import numpy as np

In [ ]:
def initialize(dim: int):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias) of type float
    """
    w: np.array = np.zeros((dim, 1))
    b: float = float(0)
    
    return w, b

In [ ]:
def sigmoid(z: np.array):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1 / (1+np.exp(-z))
    return s

In [ ]:
def propagate(w: np.array, b: float, X: np.array, Y: np.array):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    grads -- dictionary containing the gradients of the weights and bias
            (dw -- gradient of the loss with respect to w, thus same shape as w)
            (db -- gradient of the loss with respect to b, thus same shape as b)
    cost -- negative log-likelihood cost for logistic regression
    
    """
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X)+b)
    cost = -1/m * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))
    dz = A - Y
    dw = 1/m * np.dot(X, dz.T)
    db = 1/m * np.sum(dz)

    grads = {
        'dw': dw,
        'db': db
    }

    return grads, cost

    

In [ ]:
def optimize(w: np.array, b: float, X: np.array, Y, num_iterations: int=100, learning_rate: float=0.009, print_cost: bool=False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b.
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    """
    import copy
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)

    costs = []

    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)

        dw = grads['dw']
        db = grads['db']

        w = w - learning_rate * dw
        b = b - learning_rate * db

        if i % 100 == 0:
            costs.append(cost)
            if print_cost:
                print(f'Cost after iteration {i}: {cost}')

    params = {
        'w': w,
        'b': b
    }


    return params, costs

In [ ]:
def predict(w: np.array, b: float, X: np.array):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)

    A = sigmoid(np.dot(w.T, X) + b)


    for i in range(A.shape[1]):
        if A[0, i] > 0.5:
            Y_prediction[0, i] = 1

    return Y_prediction

In [ ]:
def model(X_train: np.array, Y_train: np.array, X_test: np.array, Y_test: np.array, num_iterations: int=2000, learning_rate: int=0.5, print_cost: bool=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to True to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """

    w, b = initialize(X_train.shape[0])

    params, costs = optimize(w, b , X_train, Y_train, num_iterations, learning_rate, print_cost)

    w = params['w']
    b = params['b']

    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    if print_cost:
        print(f'training accuracy: {100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100}')
        print(f'testing accuracy: {100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100}')

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
X_train = np.array([[0.1, 0.2, 0.3], [0.3, 0.4, 0.5]])
Y_train = np.array([[0, 1, 0]])
X_test = np.array([[0.2, 0.4], [0.5, 0.6]])
Y_test = np.array([[0, 1]])

def test_model():
    # Train the model and get the dictionary of results
    result = model(X_train, Y_train, X_test, Y_test, num_iterations=10, learning_rate=0.1)
    assert "costs" in result, "Result missing costs key"
    assert "Y_prediction_test" in result, "Result missing Y_prediction_test key"
    assert "Y_prediction_train" in result, "Result missing Y_prediction_train key"
    assert isinstance(result["costs"], list), "Costs should be a list"